### Imports

In [1]:
!pip install -r requirements.txt

You should consider upgrading via the '/opt/app-root/bin/python3.6 -m pip install --upgrade pip' command.


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import copy
import os

from torchvision.datasets import MNIST

from profilehooks import profile
    
import matplotlib.pylab as plt

#### Get number of threads and choose range of number of clusters

In [4]:
N_PROC = 4 #CHANGE THIS
N_TREES = 1000
N_TRAIN_CYCLES = 20

### Workload

Run a random forest classifier on the MNIST dataset. The training process is CPU only with the trees distributed across N_PROC. Note: obviously, we are not claiming a random forest classifier is the most appropriate model for an image classification problem on raw pixel features.

In [5]:
data = MNIST('./', download=True)

In [6]:
data.train_labels

/opt/app-root/lib/python3.6/site-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


tensor([5, 0, 4,  ..., 5, 6, 8])

In [7]:
#flatten each image
data_features = data.train_data.numpy().reshape(data.train_data.shape[0], -1).astype(np.float32)
data_labels = data.train_labels.numpy()
print(data_features.shape)
print(data_labels.shape)

(60000, 784)
(60000,)


/opt/app-root/lib/python3.6/site-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [8]:
model = RandomForestClassifier(n_estimators = N_TREES,
                               max_depth =15,
                               n_jobs = N_PROC,
                               oob_score = True)

### Some run times

In [9]:
@profile(immediate=True)
def train(data_features, data_labels, N_TRAIN_CYCLES):
    scores = []
    
    for _ in range(N_TRAIN_CYCLES):
        model = RandomForestClassifier(n_estimators = N_TREES,
                               max_depth =15,
                               n_jobs = N_PROC,
                               oob_score = True)
        
        model.fit(data_features, data_labels)
        
        scores.append(model.oob_score_)
    
    return scores

In [10]:
scores = train(data_features, data_labels, N_TRAIN_CYCLES)


*** PROFILER RESULTS ***
train (<ipython-input-9-0270da01e5ef>:1)
function called 1 times

         30055005 function calls (29774601 primitive calls) in 2109.621 seconds

   Ordered by: cumulative time, internal time, call count
   List reduced from 396 to 40 due to restriction <40>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.130    0.130 2109.621 2109.621 <ipython-input-9-0270da01e5ef>:1(train)
       20    0.002    0.000 2109.490  105.474 _forest.py:268(fit)
       20    0.001    0.000 1597.006   79.850 parallel.py:958(__call__)
    75460 1596.238    0.021 1596.238    0.021 {method 'acquire' of '_thread.lock' objects}
       20    0.204    0.010 1595.945   79.797 parallel.py:918(retrieve)
    20000    0.052    0.000 1595.688    0.080 pool.py:637(get)
    20000    0.034    0.000 1595.598    0.080 pool.py:634(wait)
    20140    0.097    0.000 1595.584    0.079 threading.py:533(wait)
    18715    0.145    0.000 1595.440    0.085 threading.py

In [ ]:
plt.hist(scores)